In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import scipy.io
import numpy as np

In [2]:
from matplotlib import rcParams
rcParams.update({
    "font.size": 14})

In [3]:
from scipy.interpolate import InterpolatedUnivariateSpline

In [4]:
ssa_v = np.genfromtxt('./ssa/compare_data_output.csv')[1:, :]
b_val = ssa_v[:, 0]
h_val = ssa_v[:, 1]
u_val = ssa_v[:, 2]
xh_val = ssa_v[:, 3]
xu_val = ssa_v[:, 4]

In [5]:
xh_val[-1]

2.345833636298257

In [6]:
b_int = InterpolatedUnivariateSpline(xh_val, b_val)
b_x_int = b_int.derivative(n=1)
h_int = InterpolatedUnivariateSpline(xh_val, h_val)
h_x_int = h_int.derivative(n=1)

In [7]:
def b_x(x):
    if type(x) is torch.Tensor:
        x = x.detach()
    return torch.tensor(b_x_int(x)).float()

def h_x(x):
    if type(x) is torch.Tensor:
        x = x.detach()
    return torch.tensor(h_x_int(x)).float()

def b(x):
    if type(x) is torch.Tensor:
        x = x.detach()
    return torch.tensor(b_int(x)).float()

def h(x):
    if type(x) is torch.Tensor:
        x = x.detach()
    return torch.tensor(h_int(x)).float()

In [9]:
x_b0 = torch.tensor(0).reshape(-1, 1).float()
u_b0 = torch.tensor(0).reshape(-1, 1).float()
x_b1 = torch.tensor(xh_val[-1]).reshape(-1, 1).float()
x_b1.requires_grad=True
ux_b1 = torch.tensor(19.6523087).reshape(-1, 1).float()
x = torch.tensor(xu_val, requires_grad=True).reshape(-1, 1).float()

In [10]:
# dimensional parameters
eps = 0.002587853191
c_hat = 0.8501309976
m = 3
n = 3

In [11]:
class Net(nn.Module):
    def __init__(self,
        N_in, N_out, N_hid,
        N_layers=4,
        loss_func=nn.MSELoss(),
        LEARNING_RATE = 0.001,
        optimizer = optim.Adam,
        act_func=nn.Sigmoid()
                ):
        
        super(Net, self).__init__()
        self.layers = nn.Sequential(
                nn.Linear(in_features=N_in,
                    out_features=N_hid,
                    bias=True),
                act_func,
                *(nn.Linear(in_features=N_hid,
                    out_features=N_hid,
                    bias=True), act_func)*(N_layers - 1),
                nn.Linear(in_features=N_hid,
                    out_features=N_out,
                    bias=True)
            )
        
        self.loss_func = loss_func
        self.optimizer = optimizer(self.parameters(), lr=LEARNING_RATE)
        
    def forward(self, x):
        out = self.layers(x)
        return out
    
    def PDE(self, x, h, h_x, b_x):
        # Evaluate network and derivatives
        u_hat = self.forward(x)
        u_x_hat = torch.autograd.grad(u_hat, x, torch.ones(x.shape), create_graph=True)[0]
        u_xx_hat = torch.autograd.grad(u_x_hat, x, torch.ones(x.shape), create_graph=True)[0]
        
        # Calculate terms
        prefactor = 4*eps
        extensional_term1 = h_x(x) * torch.abs(u_x_hat)**(1/3-1) * u_x_hat
        extensional_term2 = h(x)/3*torch.abs(u_x_hat)**(1/3-2)*u_xx_hat
        extensional = prefactor * (extensional_term1 + extensional_term2)
        basal = c_hat * torch.abs(u_hat)**(1/3-1)*u_hat
        driving = h(x) * (h_x(x) - b_x(x))
        
        pde = extensional - basal - driving
        return pde
    
    def loss_dirichlet(self, x_b0, u_b0):
        u_b0_hat = self.forward(x_b0)
        loss_b0 = self.loss_func(u_b0, u_b0_hat.reshape(u_b0.shape))
        return loss_b0
    
    def loss_neumann(self, x_b1, ux_b1):
        u_b1_hat = self.forward(x_b1)
        ux_b1_hat = torch.autograd.grad(u_b1_hat, x_b1, torch.ones(x_b1.shape), create_graph=True)[0]
        loss_b1 = self.loss_func(ux_b1, ux_b1_hat.reshape(ux_b1.shape))
        return loss_b1
        
        
    def loss_PDE(self, x, h, h_x, b_x):
        pde = self.PDE(x, h, h_x, b_x)
        loss = self.loss_func(pde, torch.zeros(pde.shape, requires_grad=True))
        return loss
    
    def loss(self, x, h, h_x, b_x, x_b0, x_b1, u_b0, ux_b1):
        loss = self.loss_PDE(x, h, h_x, b_x) + self.loss_dirichlet(x_b0, u_b0) + self.loss_neumann(x_b1, ux_b1)
        return loss
    
    def step(self, x, h, h_x, b_x, x_b0, x_b1, u_b0, ux_b1):
        self.train()
        self.optimizer.zero_grad()
        loss = self.loss(x, h, h_x, b_x, x_b0, x_b1, u_b0, ux_b1)
        loss.backward(retain_graph=True)
        self.optimizer.step()
        self.eval()
        return self.loss_PDE(x, h, h_x, b_x), self.loss_dirichlet(x_b0, u_b0), self.loss_neumann(x_b1, ux_b1)

In [12]:
def eval_net(net, u, x):
    u = torch.tensor(u).reshape(-1, 1)
    u_hat = net(x).detach().reshape(-1,1)
    MSE = torch.mean((u - u_hat)**2)
    return MSE

In [13]:
def progress(i, total):
    print("\r", '{:.2f}% '.format((i+1)*100/total), end="")

In [ ]:
loss_pde = []
loss_neumann = []
loss_dirichlet = []
loss_data = []
lam = []
loss_epochs = []
epochs = 20000
net = Net(N_in=1, N_out=1, N_hid=20, N_layers=2)
for e in range(epochs):
    _loss_pde, _loss_dirichlet, _loss_neumann = net.step(x, h, h_x, b_x, x_b0, x_b1, u_b0, ux_b1)
    loss_pde.append(_loss_pde.detach())
    loss_dirichlet.append(_loss_dirichlet.detach())
    loss_neumann.append(_loss_neumann.detach())

    loss_data.append(eval_net(net, u_val, x))
    progress(e, epochs)
    #if (e+1)%200 == 0:
    #    print('Epoch: {:d}. log Data loss: {:.2f}. log training loss: {:.2f}'.format(e+1, torch.log(loss_data[-1]), torch.log(sum(loss[-1]))))
        


 65.78% 

In [ ]:
fig, ax = plt.subplots(figsize=(7,7))
ax.semilogy(loss_pde, label='PDE loss')
ax.semilogy(loss_dirichlet, label='Dirichlet loss')
ax.semilogy(loss_neumann, label='Neumann loss')
ax.semilogy(loss_data, label='Evaluation loss')
ax.legend()
ax.set_ylabel('MSE')
ax.set_xlabel('Epoch')
ax.grid()
fig.tight_layout()


In [ ]:
plt.plot(xu_val, net(x).detach())
plt.plot(xu_val, u_val)

In [94]:
(u_val[-1]-u_val[-2])/(xu_val[-1]-xu_val[-2])

14.149709127114217

In [98]:
u_b1_hat = net.forward(x_b1)
ux_b1_hat = torch.autograd.grad(u_b1_hat, x_b1, torch.ones(x_b1.shape), create_graph=True)[0]

In [99]:
ux_b1_hat

tensor([[-0.0240]], grad_fn=<TBackward>)

# 